https://www.dataquest.io/blog/python-pandas-databases/

All of the data is in a SQLite database called flights.db, which contains three tables – airports, airlines, and routes. You can download the data https://www.dropbox.com/s/a2wax843eniq12g/flights.db?dl=0

In [1]:
ls

 Volume in drive C is WINDOWS
 Volume Serial Number is AE83-8D62

 Directory of C:\Users\RCole02.ROYALSURREY\Desktop\GitHub\Useful-python-for-medical-physics\Experiments in ipython notebooks\SQL

04/11/2016  09:39    <DIR>          .
04/11/2016  09:39    <DIR>          ..
04/11/2016  09:39    <DIR>          .ipynb_checkpoints
04/11/2016  09:31         5,415,936 flights.db
04/11/2016  09:38                72 sqlite3 test on airline data.ipynb
               2 File(s)      5,416,008 bytes
               3 Dir(s)  372,543,750,144 bytes free


# Quickstart

In [2]:
import sqlite3
conn = sqlite3.connect("flights.db")

In [3]:
cur = conn.cursor()

fetch the first 5 rows from the airlines table:

In [4]:
cur.execute("select * from airlines limit 5;")

run another command to actually fetch the results. We can use the fetchall method to fetch all of the results of a query. Returns a list of tuples

In [5]:
results = cur.fetchall()
print(results)

[(0, '1', 'Private flight', '\\N', '-', None, None, None, 'Y'), (1, '2', '135 Airways', '\\N', None, 'GNL', 'GENERAL', 'United States', 'N'), (2, '3', '1Time Airline', '\\N', '1T', 'RNX', 'NEXTIME', 'South Africa', 'Y'), (3, '4', '2 Sqn No 1 Elementary Flying Training School', '\\N', None, 'WYT', None, 'United Kingdom', 'N'), (4, '5', '213 Flight Unit', '\\N', None, 'TFU', None, 'Russia', 'N')]


In [25]:
cur.close()   # Close the connection
conn.close()

# Pandas

We can use the pandas read_sql_query function to read the results of a SQL query directly into a pandas DataFrame. It’s highly recommended to use the read_sql_query function when possible.

In [11]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("flights.db")
df = pd.read_sql_query("select * from airlines limit 5;", conn)
df

,index,id,name,alias,iata,icao,callsign,country,active
0,0,1,Private flight,\N,-,None,None,None,Y
1,1,2,135 Airways,\N,None,GNL,GENERAL,United States,N
2,2,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
3,3,4,2 Sqn No 1 Elementary Flying Training School,\N,None,WYT,None,United Kingdom,N
4,4,5,213 Flight Unit,\N,None,TFU,None,Russia,N


In [12]:
df["country"]

0              None
1     United States
2      South Africa
3    United Kingdom
4            Russia
Name: country, dtype: object

## Modifying databases

To insert a row, we need to write an INSERT query. The below code will add a new row to the airlines table. We specify 9 values to insert, one for each column in airlines. This will add a new row to the table.

In [13]:
cur = conn.cursor()
cur.execute("insert into airlines values (6048, 19846, 'Test flight', '', '', null, null, null, 'Y')")

If you try to query the table now, you actually won’t see the new row yet. Instead you’ll see that a file was created called flights.db-journal.  flights.db-journal is storing the new row until you’re ready to commit it to the main database, flights.db.

SQLite doesn’t write to the database until you commit a transaction. A transaction consists of 1 or more queries that all make changes to the database at once. This is designed to make it easier to recover from accidental changes, or errors. Transactions allow you to run several queries, then finally alter the database with the results of all of them. This ensures that if one of the queries fails, the database isn’t partially updated.

By default, sqlite3 opens a transaction when you do any query that modifies the database. You can read more about it here. We can commit the transaction, and add our new row to the airlines table, using the commit method:

In [14]:
conn.commit()

Now, when we query flights.db, we’ll see the extra row that contains our test flight:

In [15]:
pd.read_sql_query("select * from airlines where id=19846;", conn)

,index,id,name,alias,iata,icao,callsign,country,active
0,6048,19846,Test flight,,,None,None,None,Y


# Passing parameters into a query

In the last query, we hardcoded the values we wanted to insert into the database. Most of the time, when you insert data into a database, it won’t be hardcoded, it will be dynamic values you want to pass in. These dynamic values might come from downloaded data, or might come from user input.

When working with dynamic data, it might be tempting to insert values using Python string formatting: You want to avoid doing this! Inserting values with Python string formatting makes your program vulnerable to SQL Injection attacks. Luckily, sqlite3 has a straightforward way to inject dynamic values without relying on string formatting:

In [16]:
cur = conn.cursor()
values = ('Test Flight', 'Y')
cur.execute("insert into airlines values (6049, 19847, ?, '', '', null, null, null, ?)", values)
conn.commit()

Any ? value in the query will be replaced by a value in values. The first ? will be replaced by the first item in values, the second by the second, and so on. This works for any type of query. This created a SQLite parameterized query, which avoids SQL injection issues.

We can modify rows in a SQLite table using the execute method:

In [17]:
cur = conn.cursor()
values = ('USA', 19847)
cur.execute("update airlines set country=? where id=?", values)
conn.commit()

In [18]:
pd.read_sql_query("select * from airlines where id=19847;", conn)

,index,id,name,alias,iata,icao,callsign,country,active
0,6049,19847,Test Flight,,,None,None,USA,Y


# Creating tables with Pandas

The pandas package gives us a much faster way to create tables. We just have to create a DataFrame first, then export it to a SQL table

In [19]:
from datetime import datetime
df = pd.DataFrame(
    [[1, datetime(2016, 9, 29, 0, 0) , datetime(2016, 9, 29, 12, 0), 'T1', 1]], 
    columns=["id", "departure", "arrival", "number", "route_id"]
)
df

,id,departure,arrival,number,route_id
0,1,2016-09-29,2016-09-29 12:00:00,T1,1


Then, we’ll be able to call the to_sql method to convert df to a table in a database. We set the keep_exists parameter to replace to delete and replace any existing tables named daily_flights:

In [20]:
df.to_sql("daily_flights", conn, if_exists="replace")

We can then verify that everything worked by querying the database:

In [22]:
pd.read_sql_query("select * from daily_flights;", conn)

,index,id,departure,arrival,number,route_id
0,0,1,2016-09-29 00:00:00,2016-09-29 12:00:00,T1,1


# Altering tables with Pandas

It’s also possible to use Pandas to alter tables by exporting the table to a DataFrame, making modifications to the DataFrame, then exporting the DataFrame to a table:
This code will add a column called delay_minutes to the daily_flights table.

In [24]:
df = pd.read_sql("select * from daily_flights", conn)
df["delay_minutes"] = None
df.to_sql("daily_flights", conn, if_exists="replace")